# 🫀 Machine Learning w Medycynie - Analiza Chorób Serca

W tym notebooku poznamy 4 podstawowe techniki uczenia maszynowego:

1. 🎯 **Regresja Logistyczna** - klasyfikacja (przewidywanie kategorii)
2. 📈 **Regresja Liniowa** - przewidywanie wartości ciągłych
3. 🔵 **K-Means** - klasteryzacja (grupowanie danych)
4. 🌀 **PCA** - redukcja wymiarowości

---
**Instrukcja:** Uruchamiaj kolejne komórki przyciskiem ▶️ lub skrótem `Shift + Enter`


## 📦 Część 1: Import Bibliotek


In [ ]:
# Import bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Nasze 4 metody
from sklearn.linear_model import LogisticRegression  # Klasyfikacja
from sklearn.linear_model import LinearRegression     # Regresja
from sklearn.cluster import KMeans                    # Klasteryzacja
from sklearn.decomposition import PCA                 # Redukcja wymiarowości

# Metryki
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, r2_score, silhouette_score

# Ustawienia
plt.rcParams['figure.figsize'] = (10, 6)
import warnings
warnings.filterwarnings('ignore')

print("✅ Biblioteki załadowane!")


## 📥 Część 2: Wczytanie Danych

Prześlij plik `heart_disease_uci.csv` klikając "Upload" po lewej stronie.


In [ ]:
# Wczytanie pliku w Google Colab
from google.colab import files
uploaded = files.upload()


In [ ]:
# Wczytanie danych
df = pd.read_csv('heart_disease_uci.csv')
print(f"📊 Rozmiar: {df.shape[0]} pacjentów, {df.shape[1]} cech")
df.head()


## 🧹 Część 3: Przygotowanie Danych


In [ ]:
# Przygotowanie danych

# Usuwamy niepotrzebne kolumny
df_clean = df.drop(columns=['id', 'dataset'])

# Kodowanie zmiennych tekstowych na liczby
for col in df_clean.select_dtypes(include=['object', 'bool']).columns:
    le = LabelEncoder()
    df_clean[col] = le.fit_transform(df_clean[col].astype(str))

# Uzupełnienie brakujących wartości medianą
imputer = SimpleImputer(strategy='median')
df_imputed = pd.DataFrame(imputer.fit_transform(df_clean), columns=df_clean.columns)

# Przygotowanie zmiennych
X = df_imputed.drop(columns=['num'])  # Cechy (bez zmiennej docelowej)
y = (df_imputed['num'] > 0).astype(int)  # 0=zdrowy, 1=chory

# Podział na zbiór treningowy (80%) i testowy (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standaryzacja (ważne dla regresji logistycznej!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✅ Dane przygotowane!")
print(f"📊 Zbiór treningowy: {X_train.shape[0]} próbek")
print(f"📊 Zbiór testowy: {X_test.shape[0]} próbek")


---
# 🎯 1. REGRESJA LOGISTYCZNA (Klasyfikacja)

**Cel:** Przewidzieć czy pacjent ma chorobę serca (0=zdrowy, 1=chory)

**Jak działa:** Model uczy się na danych treningowych, a potem przewiduje klasę dla nowych danych.


In [ ]:
# KROK 1: Tworzenie i trenowanie modelu
model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train_scaled, y_train)  # Uczenie na danych treningowych

# KROK 2: Predykcja na zbiorze testowym
y_pred = model_lr.predict(X_test_scaled)

# KROK 3: Ocena modelu
accuracy = accuracy_score(y_test, y_pred)
print(f"🎯 REGRESJA LOGISTYCZNA")
print(f"   Dokładność: {accuracy:.2%}")

# Macierz pomyłek
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Zdrowy', 'Chory'], yticklabels=['Zdrowy', 'Chory'])
plt.xlabel('Predykcja')
plt.ylabel('Rzeczywistość')
plt.title('Macierz Pomyłek')
plt.show()


### 🔮 Przykład użycia .predict() - przewidywanie dla nowego pacjenta


In [ ]:
# Przykład: Przewidywanie dla NOWEGO pacjenta
# Weźmy dane pierwszego pacjenta z testu jako przykład

nowy_pacjent = X_test.iloc[[0]]  # Jeden pacjent (jako DataFrame)
print("📋 Dane nowego pacjenta:")
print(nowy_pacjent)

# Standaryzacja (musi być taka sama jak przy trenowaniu!)
nowy_pacjent_scaled = scaler.transform(nowy_pacjent)

# UŻYCIE .predict() - przewidywanie klasy
predykcja = model_lr.predict(nowy_pacjent_scaled)
print(f"\n🔮 Predykcja: {predykcja[0]} ({'CHORY' if predykcja[0] == 1 else 'ZDROWY'})")

# Prawdopodobieństwa dla każdej klasy
prawdopodobienstwa = model_lr.predict_proba(nowy_pacjent_scaled)
print(f"📊 Prawdopodobieństwo:")
print(f"   - Zdrowy: {prawdopodobienstwa[0][0]:.2%}")
print(f"   - Chory:  {prawdopodobienstwa[0][1]:.2%}")

# Sprawdźmy rzeczywistą wartość
print(f"\n✅ Rzeczywista wartość: {y_test.iloc[0]} ({'CHORY' if y_test.iloc[0] == 1 else 'ZDROWY'})")


---
# 📈 2. REGRESJA LINIOWA

**Cel:** Przewidzieć wartość ciągłą - maksymalne tętno pacjenta (thalch)

**Jak działa:** Model znajduje liniową zależność między cechami a zmienną docelową.


In [ ]:
# Przygotowanie danych do regresji
# X = cechy (bez thalch i num), y = thalch (maksymalne tętno)
X_reg = df_imputed.drop(columns=['thalch', 'num'])
y_reg = df_imputed['thalch']

# Podział danych
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Standaryzacja
scaler_reg = StandardScaler()
X_reg_train_scaled = scaler_reg.fit_transform(X_reg_train)
X_reg_test_scaled = scaler_reg.transform(X_reg_test)

# KROK 1: Tworzenie i trenowanie modelu
model_reg = LinearRegression()
model_reg.fit(X_reg_train_scaled, y_reg_train)

# KROK 2: Predykcja
y_reg_pred = model_reg.predict(X_reg_test_scaled)

# KROK 3: Ocena
r2 = r2_score(y_reg_test, y_reg_pred)
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))

print(f"📈 REGRESJA LINIOWA")
print(f"   R² (współczynnik determinacji): {r2:.3f}")
print(f"   RMSE (błąd): {rmse:.1f} bpm")
print(f"\n💡 R²={r2:.1%} oznacza, że model wyjaśnia {r2*100:.0f}% zmienności tętna")

# Wykres
plt.figure(figsize=(8, 6))
plt.scatter(y_reg_test, y_reg_pred, alpha=0.5)
plt.plot([y_reg_test.min(), y_reg_test.max()], [y_reg_test.min(), y_reg_test.max()], 'r--', label='Idealna predykcja')
plt.xlabel('Rzeczywiste tętno (bpm)')
plt.ylabel('Przewidziane tętno (bpm)')
plt.title('Regresja Liniowa - Predykcja vs Rzeczywistość')
plt.legend()
plt.show()


### 🔮 Przykład użycia .predict() - przewidywanie tętna dla nowego pacjenta


In [ ]:
# Przykład: Przewidywanie tętna dla NOWEGO pacjenta

nowy_pacjent_reg = X_reg_test.iloc[[0]]  # Jeden pacjent
print("📋 Dane nowego pacjenta:")
print(nowy_pacjent_reg[['age', 'sex', 'trestbps', 'chol']].T)  # Tylko kilka cech dla czytelności

# Standaryzacja
nowy_pacjent_reg_scaled = scaler_reg.transform(nowy_pacjent_reg)

# UŻYCIE .predict() - przewidywanie wartości ciągłej
predykcja_tetno = model_reg.predict(nowy_pacjent_reg_scaled)
print(f"\n🔮 Przewidziane tętno: {predykcja_tetno[0]:.0f} bpm")
print(f"✅ Rzeczywiste tętno: {y_reg_test.iloc[0]:.0f} bpm")
print(f"📊 Błąd: {abs(predykcja_tetno[0] - y_reg_test.iloc[0]):.1f} bpm")


---
# 🔵 3. K-MEANS (Klasteryzacja)

**Cel:** Pogrupować pacjentów w podobne grupy BEZ użycia etykiet (uczenie nienadzorowane)

**Jak działa:** Algorytm szuka K centrów klastrów i przypisuje każdy punkt do najbliższego centrum.


In [ ]:
# Przygotowanie danych do klasteryzacji
X_cluster = df_imputed.drop(columns=['num'])
X_cluster_scaled = StandardScaler().fit_transform(X_cluster)

#╔═══════════════════════════════════════════════════════════════╗
#║  🎛️ PARAMETR DO ZMIANY: liczba klastrów                      ║
#╚═══════════════════════════════════════════════════════════════╝
N_CLUSTERS = 3  # 🔧 Zmień: 2, 3, 4, 5

# KROK 1: Tworzenie i trenowanie modelu
model_kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
model_kmeans.fit(X_cluster_scaled)

# Etykiety klastrów
cluster_labels = model_kmeans.labels_

# Ocena
silhouette = silhouette_score(X_cluster_scaled, cluster_labels)

print(f"🔵 K-MEANS")
print(f"   Liczba klastrów: {N_CLUSTERS}")
print(f"   Silhouette Score: {silhouette:.3f} (im bliżej 1, tym lepiej)")
print(f"\n📊 Rozmiary klastrów:")
for i in range(N_CLUSTERS):
    print(f"   Klaster {i}: {(cluster_labels == i).sum()} pacjentów")


In [ ]:
# Wizualizacja klastrów (na 2 wybranych cechach)
plt.figure(figsize=(10, 6))
scatter = plt.scatter(df_imputed['age'], df_imputed['thalch'], c=cluster_labels, cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Klaster')
plt.xlabel('Wiek')
plt.ylabel('Maksymalne tętno')
plt.title(f'K-Means: Podział na {N_CLUSTERS} klastrów')
plt.show()


### 🔮 Przykład użycia .predict() - przypisanie nowego pacjenta do klastra


In [ ]:
# Przykład: Przypisanie NOWEGO pacjenta do klastra

# Weźmy kilku pacjentów jako przykład
nowi_pacjenci = X_cluster.iloc[[0, 100, 500]]  # 3 różnych pacjentów
print("📋 Dane 3 nowych pacjentów (wybrane cechy):")
print(nowi_pacjenci[['age', 'sex', 'trestbps', 'chol', 'thalch']])

# Standaryzacja (używamy tego samego scalera co przy fit!)
scaler_cluster = StandardScaler()
scaler_cluster.fit(X_cluster)  # fit na całych danych
nowi_pacjenci_scaled = scaler_cluster.transform(nowi_pacjenci)

# UŻYCIE .predict() - przypisanie do klastra
przypisane_klastry = model_kmeans.predict(nowi_pacjenci_scaled)

print(f"\n🔮 Przypisane klastry:")
for i, klaster in enumerate(przypisane_klastry):
    print(f"   Pacjent {i+1}: Klaster {klaster}")


---
# 🌀 4. PCA (Redukcja Wymiarowości)

**Cel:** Zredukować liczbę wymiarów (cech) zachowując jak najwięcej informacji

**Jak działa:** PCA znajduje nowe osie (główne składowe), które maksymalizują wariancję danych.


In [ ]:
# Przygotowanie danych
X_pca_data = df_imputed.drop(columns=['num'])
X_pca_scaled = StandardScaler().fit_transform(X_pca_data)

print(f"📊 Oryginalna liczba wymiarów: {X_pca_data.shape[1]}")

# KROK 1: Tworzenie i trenowanie modelu PCA (redukcja do 2 wymiarów)
model_pca = PCA(n_components=2)
model_pca.fit(X_pca_scaled)

# KROK 2: Transformacja danych
X_pca_transformed = model_pca.transform(X_pca_scaled)

print(f"📊 Nowa liczba wymiarów: {X_pca_transformed.shape[1]}")

# Wyjaśniona wariancja
explained_var = model_pca.explained_variance_ratio_
print(f"\n📊 Wyjaśniona wariancja:")
print(f"   PC1: {explained_var[0]:.1%}")
print(f"   PC2: {explained_var[1]:.1%}")
print(f"   Suma: {sum(explained_var):.1%}")


In [ ]:
# Wizualizacja PCA
y_binary = (df_imputed['num'] > 0).astype(int)
colors = ['#2ecc71' if label == 0 else '#e74c3c' for label in y_binary]

plt.figure(figsize=(10, 6))
plt.scatter(X_pca_transformed[:, 0], X_pca_transformed[:, 1], c=colors, alpha=0.6)
plt.xlabel(f'PC1 ({explained_var[0]:.1%} wariancji)')
plt.ylabel(f'PC2 ({explained_var[1]:.1%} wariancji)')
plt.title('PCA - Dane zredukowane do 2 wymiarów')

# Legenda
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='#2ecc71', label='Zdrowy'),
                   Patch(facecolor='#e74c3c', label='Chory')]
plt.legend(handles=legend_elements)
plt.show()


### 🔮 Przykład użycia .transform() - transformacja nowego pacjenta

**Uwaga:** PCA używa `.transform()` zamiast `.predict()` - transformuje dane do nowej przestrzeni.


In [ ]:
# Przykład: Transformacja NOWEGO pacjenta do przestrzeni PCA

nowy_pacjent_pca = X_pca_data.iloc[[0]]  # Jeden pacjent
print(f"📋 Oryginalne dane pacjenta: {X_pca_data.shape[1]} cech")
print(nowy_pacjent_pca[['age', 'sex', 'trestbps', 'chol', 'thalch']].T)

# Standaryzacja (używamy tego samego scalera!)
scaler_pca = StandardScaler()
scaler_pca.fit(X_pca_data)
nowy_pacjent_pca_scaled = scaler_pca.transform(nowy_pacjent_pca)

# UŻYCIE .transform() - transformacja do nowej przestrzeni
nowe_wspolrzedne = model_pca.transform(nowy_pacjent_pca_scaled)

print(f"\n🔮 Nowe współrzędne (2 wymiary):")
print(f"   PC1: {nowe_wspolrzedne[0][0]:.3f}")
print(f"   PC2: {nowe_wspolrzedne[0][1]:.3f}")

# Zaznaczenie na wykresie
plt.figure(figsize=(10, 6))
plt.scatter(X_pca_transformed[:, 0], X_pca_transformed[:, 1], c=colors, alpha=0.3, label='Wszyscy pacjenci')
plt.scatter(nowe_wspolrzedne[0][0], nowe_wspolrzedne[0][1], c='blue', s=200, marker='*', label='Nowy pacjent')
plt.xlabel(f'PC1')
plt.ylabel(f'PC2')
plt.title('PCA - Pozycja nowego pacjenta')
plt.legend()
plt.show()


---
# 📝 PODSUMOWANIE


In [ ]:
print("="*60)
print("📋 PODSUMOWANIE METOD")
print("="*60)

print("""
🎯 1. REGRESJA LOGISTYCZNA (Klasyfikacja)
   - Przewiduje KATEGORIĘ (0 lub 1)
   - Używa: model.predict(X) → zwraca etykietę klasy
   - Dodatkowo: model.predict_proba(X) → prawdopodobieństwa

📈 2. REGRESJA LINIOWA
   - Przewiduje WARTOŚĆ CIĄGŁĄ (np. 120.5)
   - Używa: model.predict(X) → zwraca wartość liczbową

🔵 3. K-MEANS (Klasteryzacja)
   - Przypisuje punkty do KLASTRÓW (grup)
   - Używa: model.predict(X) → zwraca numer klastra

🌀 4. PCA (Redukcja wymiarowości)
   - TRANSFORMUJE dane do mniejszej liczby wymiarów
   - Używa: model.transform(X) → zwraca nowe współrzędne
""")

print("="*60)
print(f"📊 WYNIKI:")
print(f"   Regresja Logistyczna: {accuracy:.2%} dokładności")
print(f"   Regresja Liniowa: R² = {r2:.3f}")
print(f"   K-Means: Silhouette = {silhouette:.3f}")
print(f"   PCA: {sum(explained_var):.1%} wyjaśnionej wariancji")
print("="*60)


---
## 🎮 DO SAMODZIELNEGO EKSPERYMENTOWANIA

1. **K-Means:** Zmień `N_CLUSTERS` (2, 3, 4, 5) i zaobserwuj jak zmieniają się klastry

2. **Regresja:** Spróbuj przewidzieć inną zmienną (np. `chol` zamiast `thalch`)

3. **Predict:** Zmień numer pacjenta w przykładach `.predict()` i zobacz inne wyniki
